<a href="https://colab.research.google.com/github/YannMaa/traco_2024/blob/main/solution_yannis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("test")

test


Setup

In [1]:
!pip install ultralytics
!pip install deep-sort-realtime
!pip install opencv-python


In [40]:
# # preprocessing
# import os
# import cv2
# import pandas as pd
# import json
# import shutil

# # Funktion zum Extrahieren von Frames mit eindeutigen Namen
# def extract_frames(video_path, image_folder, video_id):
#     cap = cv2.VideoCapture(video_path)
#     frame_id = 0
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Eindeutiger Bildname pro Video und Frame
#         image_path = os.path.join(image_folder, f"{video_id}_frame_{frame_id}.jpg")
#         cv2.imwrite(image_path, frame)
#         frame_id += 1
#     cap.release()
#     print(f"Frames aus {video_path} gespeichert unter {image_folder}")

# # Funktion zur Berechnung der normalisierten Bounding Box
# def convert_to_yolo_format(x, y, image_width, image_height):
#     width = 50
#     height = 50
#     x_center = x / image_width
#     y_center = y / image_height
#     width_norm = width / image_width
#     height_norm = height / image_height
#     return x_center, y_center, width_norm, height_norm

# # Verarbeitung einer CSV- und .traco-Datei für ein Video
# def process_files(csv_file, traco_file, image_folder, label_folder, video_id):
#     df = pd.read_csv(csv_file)
#     with open(traco_file, 'r') as f:
#         traco_data = json.load(f)

#     for i, row in df.iterrows():
#         frame_id = int(row['t'])
#         x, y = row['x'], row['y']
#         image_path = os.path.join(image_folder, f"{video_id}_frame_{frame_id}.jpg")
#         image = cv2.imread(image_path)
#         if image is None:
#             continue
#         image_height, image_width, _ = image.shape
#         x_center, y_center, width_norm, height_norm = convert_to_yolo_format(x, y, image_width, image_height)
#         label_file = os.path.join(label_folder, f"{video_id}_frame_{frame_id}.txt")
#         with open(label_file, 'a') as f:
#             f.write(f"0 {x_center} {y_center} {width_norm} {height_norm}\n")

#     for roi in traco_data['rois']:
#         frame_id = roi['z']
#         x, y = roi['pos']
#         image_path = os.path.join(image_folder, f"{video_id}_frame_{frame_id}.jpg")
#         image = cv2.imread(image_path)
#         if image is None:
#             continue
#         image_height, image_width, _ = image.shape
#         x_center, y_center, width_norm, height_norm = convert_to_yolo_format(x, y, image_width, image_height)
#         label_file = os.path.join(label_folder, f"{video_id}_frame_{frame_id}.txt")
#         with open(label_file, 'a') as f:
#             f.write(f"0 {x_center} {y_center} {width_norm} {height_norm}\n")

# # Hauptfunktion zur Verarbeitung ALLER Videos im Verzeichnis
# def process_all_training_data(folder_path):
#     image_folder = os.path.join(folder_path, 'images')
#     label_folder = os.path.join(folder_path, 'labels')

#     # Alte Inhalte löschen (optional, aber empfohlen)
#     for folder in [image_folder, label_folder]:
#         if os.path.exists(folder):
#             shutil.rmtree(folder)
#         os.makedirs(folder)

#     # Alle Videodateien im Ordner durchgehen
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.mp4'):
#             video_path = os.path.join(folder_path, filename)
#             video_id = os.path.splitext(filename)[0]  # z. B. "training01"

#             csv_file = os.path.join(folder_path, f"{video_id}.csv")
#             traco_file = os.path.join(folder_path, f"{video_id}.traco")
#             if not os.path.exists(csv_file) or not os.path.exists(traco_file):
#                 print(f"Fehlende Dateien für {video_id}, überspringe...")
#                 continue

#             print(f"Verarbeite {video_id}...")
#             extract_frames(video_path, image_folder, video_id)
#             process_files(csv_file, traco_file, image_folder, label_folder, video_id)

# # STARTE den Prozess
# process_all_training_data('/content/drive/MyDrive/traco_2024/training')


Verarbeite training01...
Frames aus /content/drive/MyDrive/traco_2024/training/training01.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite training0100...
Frames aus /content/drive/MyDrive/traco_2024/training/training0100.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite training021...
Frames aus /content/drive/MyDrive/traco_2024/training/training021.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite training019...
Frames aus /content/drive/MyDrive/traco_2024/training/training019.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite training012...
Frames aus /content/drive/MyDrive/traco_2024/training/training012.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite training018...
Frames aus /content/drive/MyDrive/traco_2024/training/training018.mp4 gespeichert unter /content/drive/MyDrive/traco_2024/training/images
Verarbeite train

In [2]:
# train & validation split
import os
import random
import shutil

# Pfade definieren
images_dir = "/content/drive/MyDrive/traco_2024/training/images"
labels_dir = "/content/drive/MyDrive/traco_2024/training/labels"
video_dir = "/content/drive/MyDrive/traco_2024/training"

train_images_out = "/content/drive/MyDrive/traco_2024/training/model_training/images"
train_labels_out = "/content/drive/MyDrive/traco_2024/training/model_training/labels"
val_images_out = "/content/drive/MyDrive/traco_2024/training/model_validation/images"
val_labels_out = "/content/drive/MyDrive/traco_2024/training/model_validation/labels"

# Zielordner erstellen, wenn sie noch nicht existieren
for d in [train_images_out, train_labels_out, val_images_out, val_labels_out]:
    os.makedirs(d, exist_ok=True)

# Alle Video-Dateien auflisten (angenommen: es gibt Bild- und Label-Dateien mit den gleichen Namen wie die Videos)
video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")]  # Liste der Video-Dateien

# Video-IDs extrahieren und sicherstellen, dass wir den Unterstrich anhängen
video_ids = list(set(os.path.splitext(f)[0] for f in video_files))  # Video-IDs extrahieren

# Füge '_' an jede ID an
video_ids = [vid_id + '_' for vid_id in video_ids]  # Unterstrich zu jeder ID hinzufügen


# Zufällig 20 Video-IDs auswählen
random.shuffle(video_ids)
selected_video_ids = video_ids[:20]

print(f"Ausgewählte Video-IDs: {selected_video_ids}")

# Für jedes ausgewählte Video die entsprechenden Bilder und Labels kopieren
train_image_files = []
train_label_files = []
val_image_files = []
val_label_files = []

# Gehe durch jede ausgewählte Video-ID
for video_id in selected_video_ids:
    # Bild- und Label-Dateien für das Video finden
    img_files = [f for f in os.listdir(images_dir) if f.startswith(video_id) and (f.endswith(".jpg") or f.endswith(".png"))]
    label_files = [f for f in os.listdir(labels_dir) if f.startswith(video_id) and f.endswith(".txt")]

    # Sicherstellen, dass wir genau 100 Bilder und 100 Labels haben
    if len(img_files) != 101 or len(label_files) != 101:
        print(f"Warnung: Für Video-ID {video_id} fehlen Bilder oder Labels!")
        print(len(img_files))
        print((len(label_files)))
        continue

    # Die Bild- und Label-Dateien nach der Reihenfolge sortieren
    img_files.sort()
    label_files.sort()

    print(video_id)
    print(img_files)
    print(len(img_files[:101]))

    # Slicing der ersten X Dateien für die Validierung (z. B. 20 Bilder/Labels für die Validierung)
    val_image_files.extend(img_files)
    val_label_files.extend(label_files)



# Der Rest geht ins Training
# Train-Dateien durch Subtraktion bestimmen
all_image_files = set(os.listdir(images_dir))
all_label_files = set(os.listdir(labels_dir))

val_image_files_set = set(val_image_files)
val_label_files_set = set(val_label_files)

train_image_files = list(all_image_files - val_image_files_set)
train_label_files = list(all_label_files - val_label_files_set)


# Funktion zum Kopieren der Dateien in die Zielordner
def copy_files(file_list, src_dir, dst_dir):
    for file in file_list:
        src_path = os.path.join(src_dir, file)
        dst_path = os.path.join(dst_dir, file)

        # Sicherstellen, dass die Datei existiert
        if not os.path.exists(src_path):
            print(f"Fehler: Datei nicht gefunden - {src_path}")
            continue

        shutil.copy(src_path, dst_path)

# Kopiere die Bild- und Label-Dateien in den jeweiligen Ordner
copy_files(train_image_files, images_dir, train_images_out)
copy_files(train_label_files, labels_dir, train_labels_out)
copy_files(val_image_files, images_dir, val_images_out)
copy_files(val_label_files, labels_dir, val_labels_out)

print(f"Trainingsset: {len(train_image_files)} Bilder, {len(train_label_files)} Labels")
print(f"Validierungsset: {len(val_image_files)} Bilder, {len(val_label_files)} Labels")






Ausgewählte Video-IDs: ['training063_', 'training097_', 'training052_', 'training092_', 'training084_', 'training077_', 'training046_', 'training065_', 'training068_', 'training08_', 'training020_', 'training012_', 'training059_', 'training066_', 'training018_', 'training088_', 'training019_', 'training013_', 'training078_', 'training050_']
training063_
['training063_frame_0.jpg', 'training063_frame_1.jpg', 'training063_frame_10.jpg', 'training063_frame_100.jpg', 'training063_frame_11.jpg', 'training063_frame_12.jpg', 'training063_frame_13.jpg', 'training063_frame_14.jpg', 'training063_frame_15.jpg', 'training063_frame_16.jpg', 'training063_frame_17.jpg', 'training063_frame_18.jpg', 'training063_frame_19.jpg', 'training063_frame_2.jpg', 'training063_frame_20.jpg', 'training063_frame_21.jpg', 'training063_frame_22.jpg', 'training063_frame_23.jpg', 'training063_frame_24.jpg', 'training063_frame_25.jpg', 'training063_frame_26.jpg', 'training063_frame_27.jpg', 'training063_frame_28.jpg', '

In [ ]:
# create .yaml file

import yaml

# Pfade zu deinen Trainings- und Validierungsbildern
train = '/content/drive/MyDrive/traco_2024/training/images'
val = '/content/drive/MyDrive/traco_2024/training/images'  # Kann gleich sein, wenn kein extra val-Set

# Klassenbezeichner – nur 1 Klasse "hexbug"
names = {0: 'hexbug'}

# Erstelle das YAML-Dictionary
dataset_yaml = {
    'train': train,
    'val': val,
    'nc': len(names),
    'names': list(names.values())  # YOLO erwartet eine Liste, nicht ein Dict
}

# Speichere YAML-Datei
yaml_path = "/content/drive/MyDrive/traco_2024/training/dataset.yaml"
with open(yaml_path, 'w') as file:
    yaml.dump(dataset_yaml, file, sort_keys=False)

print(f"YAML-Datei wurde gespeichert unter: {yaml_path}")


In [29]:
from ultralytics import YOLO
import os
import torch

# Lade YOLOv8 Modell (wir verwenden das vortrainierte Modell, du kannst es später auch mit deinem Datensatz trainieren)
model = YOLO("yolov8n.pt")  # Lade das vortrainierte YOLOv8-Modell

# Trainiere das Modell mit eigenen Daten
# Stelle sicher, dass du die korrekte dataset.yaml-Datei erstellst
dataset_yaml = "/content/drive/MyDrive/traco_2024/training/dataset.yaml"  # Pfad zur dataset.yaml-Datei

# Trainiere das Modell (z. B. für 50 Epochen)
print(f"CUDA verfügbar: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Kein GPU vorhanden'}")
model.train(data=dataset_yaml, epochs=50)

# Nach dem Training kannst du das Modell speichern
model.save('/content/drive/MyDrive/traco_2024/yolov8_model')  # Speicherort für das trainierte Modell



In [32]:
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
from matplotlib import pyplot as plt

# Initialisiere DeepSORT Tracker
deepsort = DeepSort(max_age=30, n_init=3, nn_budget=100)

# Lade das YOLOv8-Modell
model = YOLO("yolov8n.pt")  # Vortrainiertes Modell oder dein eigenes

# Video einlesen
cap = cv2.VideoCapture("/content/drive/My Drive/traco_2024/training/training01.mp4")
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fc = 0

while fc < frameCount:
    ret, frame = cap.read()

    if not ret:
        break

    # YOLOv8 Vorhersagen
    results = model(frame)  # YOLOv8 liefert ein Ergebnis, das bounding boxes und Klassen liefert

    # Extrahiere die Ergebnisse (bounding boxes, scores, etc.)
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Format: [x1, y1, x2, y2]
    confidences = results[0].boxes.conf.cpu().numpy()  # Konfidenzwerte der Vorhersage
    class_ids = results[0].boxes.cls.cpu().numpy()  # Klassen IDs

    # Extrahiere die Bounding Boxes für DeepSORT
    detections = []
    for box, conf, cls in zip(boxes, confidences, class_ids):
        detection = [box[0], box[1], box[2], box[3], conf]  # [x1, y1, x2, y2, confidence]
        detections.append(detection)

    # DeepSORT-Verfolgung
    trackers = deepsort.update_tracks(detections, frame=frame)

    # Zeichne die Verfolgungs-Ergebnisse (Tracking und Bounding Boxes)
    for track in trackers:
        track_id = track[1]
        x1, y1, x2, y2 = track[0]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Zeige das Bild mit den getrackten Objekten
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    fc += 1

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 apple, 226.0ms
Speed: 13.9ms preprocess, 226.0ms inference, 21.1ms postprocess per image at shape (1, 3, 480, 640)


TypeError: object of type 'numpy.float32' has no len()

In [31]:
from sklearn.metrics import precision_recall_curve

# Beispiel für Vorhersagen und Wahrheitswerte (Ground Truth)
y_true = []  # Liste der tatsächlichen Labels (z. B. Hexbug-ID's)
y_pred = []  # Liste der Vorhersagen (z. B. vorhersagbare Bounding Boxes)

# Berechne den Precision-Recall-Score
precision, recall, thresholds = precision_recall_curve(y_true, y_pred)

# Zeige den Score
print(f"Precision: {precision[-1]}, Recall: {recall[-1]}")


ValueError: y_true takes value in {} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.

In [ ]:
import csv

# Speichern der Tracking-Daten in einer CSV-Datei
with open('tracking_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "Object_ID", "x1", "y1", "x2", "y2", "Confidence"])

    for frame_number, trackers in enumerate(trackers):
        for track in trackers:
            track_id = track[1]
            x1, y1, x2, y2 = track[0]
            confidence = track[4]
            writer.writerow([frame_number, track_id, x1, y1, x2, y2, confidence])


In [ ]:
import json

# Speichern der Tracking-Daten in einer JSON-Datei
tracking_data = {"tracks": []}

for frame_number, trackers in enumerate(trackers):
    for track in trackers:
        track_data = {
            "frame": frame_number,
            "track_id": track[1],
            "bounding_box": [track[0][0], track[0][1], track[0][2], track[0][3]],
            "confidence": track[4]
        }
        tracking_data["tracks"].append(track_data)

with open('tracking_results.json', 'w') as json_file:
    json.dump(tracking_data, json_file)
